# Schelling's Model of Segregation

An example of an agent-based model created using the Global Open Simulator (GOS).

**This is a work in progress.**

The model uses cellular automata.

```biblatex
@article{schelling1971dynamic,
  title={Dynamic models of segregation},
  author={Schelling, Thomas C},
  journal={Journal of mathematical sociology},
  volume={1},
  number={2},
  pages={143--186},
  year={1971},
  publisher={Taylor \& Francis}
}
```

In [ ]:
%cd ~/research/migration/GOS/

import gos
import pandas as pd
import numpy as np
from collections import Counter
from random import randint, shuffle

# % matplotlib inline
import matplotlib.pylab as plt

from multiprocessing import Queue

from IPython.display import HTML

plt.style.use('ggplot')

In [ ]:
AGENT_NUM = 1000
GRID_SIZE = (32, 32)
THRESHOLD = float(3 + 1) / 9

In [ ]:
grid_x, grid_y = np.mgrid[0:GRID_SIZE[0]:1,0:GRID_SIZE[1]:1]

blank = pd.DataFrame({
        'x': np.ndarray.flatten(grid_x),
        'y': np.ndarray.flatten(grid_y),
    })

# These will be the spaces on the grid which don't have agents.
remove = np.random.choice(
    np.arange(GRID_SIZE[0]*GRID_SIZE[1]),
    GRID_SIZE[0]*GRID_SIZE[1] - AGENT_NUM,
    replace=False
)
# initialize agents on the grid.
data = pd.DataFrame({
    'type': np.random.randint(0, 2, AGENT_NUM),
    'x': np.delete(np.ndarray.flatten(grid_x), remove),
    'y': np.delete(np.ndarray.flatten(grid_y), remove),
})
data[0:5]

In [ ]:
def get_grid_vals():
    return pd.merge(blank, data, on=['x', 'y'], how='left')

def get_grid():
    return get_grid_vals()["type"].values.reshape((GRID_SIZE[0], GRID_SIZE[1]))

def neighbors(x, y):
    d = 1
    x1 = x-d
    y1 = y-d
    if x1 < 0:
        x1 = 0
    if y1 < 0:
        y1 = 0
    return grid[x1:x+d+1, y1:y+d+1].flatten()

def happy(row):
    type = row["type"]
    near = [x for x in neighbors(row["x"], row["y"]) if np.isfinite(x)]
    near = Counter(near)
    if type in near:
        return near[type] > sum(near.values()) * THRESHOLD
    return False

data['happy'] = None
grid = get_grid()


In [ ]:
def get_open():
    g = get_grid_vals()
    return [tuple(x) for x in g[g["type"].isnull()][["x", "y"]].values]

fig2 = plt.figure(figsize=(5,5))

def interate():
    grid = get_grid()
    q = get_open()

    for i, row in data[data["happy"] == False].iterrows():
        x, y = q.pop()
        q.insert(randint(0, len(q)), (row["x"], row["y"]))
        data.loc[i, 'x'] = int(x)
        data.loc[i, 'y'] = int(y)

    # Split this back into two processes. 
    for i, row in data.iterrows():
        data.loc[i, 'happy'] = happy(row)
    
    #print(len(data[data['happy'] == True]) / len(data[data['happy'].notnull()]))
    return get_grid()

In [ ]:
import matplotlib.animation as animation

ims = []
for _ in range(60):
    img = plt.imshow(interate(), animated=True)
    ims.append([img])

im_ani = animation.ArtistAnimation(fig2, ims, interval=100, repeat_delay=1000, blit=True)

HTML(im_ani.to_jshtml())